In [ ]:
def testing(X, y, X_holdout = None, y_holdout = None):
    if(X_holdout != None and y_holdout != None):
        train_labels = np.arange(X.shape[0])
        test_labels = np.arange(X_holdout.shape[0]) + X.shape[0]
        X = np.concatenate((X, X_holdout), axis = 0)
        y = np.concatenate((y, y_holdout), axis = 0)
        cv_custom = [(train_labels, test_labels)]
    estimators = [
            ('scale_predictors', StandardScaler()),
            #('feature_selector', LinearSVC(penalty='l1', dual=False)),
            #('feature_selector', SelectKBest(score_func=f_classif)),
            #('linearSVC', LinearSVC())
            ('randomforests', RandomForestClassifier())
            ]
    clf = Pipeline(estimators)
    params = dict(
            #linearSVC__C=[0.1, 1, 10],
            randomforests__max_depth=[5, 10, None], 
            randomforests__n_estimators=[10, 50, 100,1000], 
            #feature_selector__C=[0.1, 1, 10]
            #feature_selector__score_func=[chi2],
            #feature_selector__k=[5, 10, 'all'] 
            )
    if(X_holdout != None and y_holdout != None):
        grid_search = GridSearchCV(clf, param_grid=params, cv=cv_custom, scoring = 'roc_auc', n_jobs = 6)
    else:
        grid_search = GridSearchCV(clf, param_grid=params, scoring = 'roc_auc', n_jobs = 6)
    grid_search.fit(X, y)
    return grid_search

In [ ]:
def performance(results, param1, param2):
    param1_vals = [x.parameters[param1] for x in results.grid_scores_]
    param2_vals = [x.parameters[param2] for x in results.grid_scores_]
    means = [x.mean_validation_score for x in results.grid_scores_]
    df = pd.DataFrame(zip(param1_vals, param2_vals, means), columns = [param1, param2, 'means'])
    df.fillna('None', inplace=True)
    return pd.pivot_table(df, values = 'means' , index = param1, columns = param2)